# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bardai,11.6500,7.2333,34.50,13,96,3.81,NG,1699622119
1,1,jamestown,42.0970,-79.2353,8.08,68,0,3.13,US,1699622021
2,2,dudinka,69.4058,86.1778,-7.40,93,100,12.07,RU,1699622120
3,3,stanley,54.8680,-1.6985,8.07,74,20,4.63,GB,1699621942
4,4,bethel,41.3712,-73.4140,9.72,67,40,3.09,US,1699621975


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

min_temp_s = city_data_df["Max Temp"] > 21
max_temp_s = city_data_df["Max Temp"] < 27
wind_speed_s = city_data_df["Wind Speed"] > 5

In [7]:
print(sum(min_temp_s))
print(sum(max_temp_s))
print(sum(wind_speed_s))

284
432
163


In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp_df = city_data_df[min_temp_s & max_temp_s & wind_speed_s]
city_temp_df

# Drop any rows with null values
clean_city_temp_df = city_temp_range_df.dropna()
clean_city_temp_df


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
hotel_df.head()


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,bardai,NG,34.50,11.6500,7.2333,
1,jamestown,US,8.08,42.0970,-79.2353,
2,dudinka,RU,-7.40,69.4058,86.1778,
3,stanley,GB,8.07,54.8680,-1.6985,
4,bethel,US,9.72,41.3712,-73.4140,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key     
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"] 
    lat = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"

#create a params dictionary
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key     
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"] 
    lat = hotel_df.loc[index, "Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    print(name_address.url)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=4b247b82ce3b499282de9481d0734070&filter=circle%3A7.2333%2C11.65%2C10000&bias=proximity%3A7.2333%2C11.65
bardai - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=4b247b82ce3b499282de9481d0734070&filter=circle%3A-79.2353%2C42.097%2C10000&bias=proximity%3A-79.2353%2C42.097
jamestown - nearest hotel: DoubleTree Jamestown
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=4b247b82ce3b499282de9481d0734070&filter=circle%3A86.1778%2C69.4058%2C10000&bias=proximity%3A86.1778%2C69.4058
dudinka - nearest hotel: Виктория
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=4b247b82ce3b499282de9481d0734070&filter=circle%3A-1.6985%2C54.868%2C10000&bias=proximity%3A-1.6985%2C54.868
stanley - nearest hotel: Hotel 52
https://api.geoapify.com/v2/places?categories

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,bardai,NG,34.50,11.6500,7.2333,No hotel found
1,jamestown,US,8.08,42.0970,-79.2353,DoubleTree Jamestown
2,dudinka,RU,-7.40,69.4058,86.1778,Виктория
3,stanley,GB,8.07,54.8680,-1.6985,Hotel 52
4,bethel,US,9.72,41.3712,-73.4140,Hampton Inn Danbury
...,...,...,...,...,...,...
577,lucaya,BS,24.96,26.5333,-78.6667,No hotel found
578,shihezi,CN,6.04,44.3000,86.0333,石河子宾馆
579,kalabo,ZM,30.25,-14.9700,22.6814,Golden Lodge Luxury Acomodation
580,reynosa,MX,19.01,26.0833,-98.2833,Hotel Gran Premier


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)